# Modelagem Avançada — Predição de Churn

Neste notebook são avaliados modelos de maior complexidade
(Random Forest e XGBoost) para predição de churn em clientes ERP.

Os resultados são comparados com o modelo baseline de Regressão
Logística, com foco nas métricas:
- ROC-AUC
- Recall da classe positiva (churn)

O objetivo é identificar ganhos de performance e capacidade
preditiva em relação ao baseline.

-----------------------

## **1. Importações**

In [22]:
import sys
import os
import warnings
import pandas as pd

warnings.filterwarnings('ignore')

sys.path.append(os.path.abspath("../src"))

from feature_engineering import preparar_features

------------------------

## **2. Carregar dados e preparar features**

In [23]:
df = pd.read_csv("../data/raw/telco_churn.csv")

X, y, num_cols, cat_cols = preparar_features(df)

print("Variáveis numéricas:", num_cols)
print("Variáveis categóricas:", cat_cols)

Variáveis numéricas: ['seniorcitizen', 'tenure', 'monthlycharges', 'totalcharges']
Variáveis categóricas: ['gender', 'partner', 'dependents', 'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod']


------------------------

## **3. Split (igual ao baseline)**

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

------------------------

## **4. Preprocessamento (Pipeline)**

In [25]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

preprocessador = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', drop='first'), cat_cols)
    ]
)

------------------------
## **5. Modelo Random Forest**

### 5.1 Definição do modelo

In [26]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=10,
    min_samples_leaf=20,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

### 5.2 Pipeline Random Forest

In [27]:
from sklearn.pipeline import Pipeline

pipeline_rf = Pipeline(
    steps=[
        ('preprocessamento', preprocessador),
        ('modelo', rf_model)
    ]
)


### 5.3 Treinamento

In [28]:
pipeline_rf.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessamento', ...), ('modelo', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transforme

### 5.4 Avaliação

In [29]:
from sklearn.metrics import roc_auc_score, classification_report

y_proba_rf = pipeline_rf.predict_proba(X_test)[:, 1]
y_pred_rf = pipeline_rf.predict(X_test)

print(classification_report(y_test, y_pred_rf))
print("ROC-AUC Random Forest:", roc_auc_score(y_test, y_proba_rf))

              precision    recall  f1-score   support

           0       0.91      0.73      0.81      1291
           1       0.52      0.80      0.63       467

    accuracy                           0.75      1758
   macro avg       0.72      0.77      0.72      1758
weighted avg       0.81      0.75      0.76      1758

ROC-AUC Random Forest: 0.8422052191336165


------------------------
## **6. Modelo XGBOOST**

### 6.1 Definição do modelo

In [30]:
from xgboost import XGBClassifier

scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]

xgb_model = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    eval_metric='logloss',
    random_state=42
)

### 6.2 Pipeline XGBoost

In [31]:
pipeline_xgb = Pipeline(
    steps=[
        ('preprocessamento', preprocessador),
        ('modelo', xgb_model)
    ]
)

### 6.3 Treinamento

In [32]:
pipeline_xgb.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessamento', ...), ('modelo', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transforme

### 6.4 Avaliação

In [33]:
y_proba_xgb = pipeline_xgb.predict_proba(X_test)[:, 1]
y_pred_xgb = pipeline_xgb.predict(X_test)

print(classification_report(y_test, y_pred_xgb))
print("ROC-AUC XGBoost:", roc_auc_score(y_test, y_proba_xgb))

              precision    recall  f1-score   support

           0       0.89      0.77      0.83      1291
           1       0.54      0.72      0.62       467

    accuracy                           0.76      1758
   macro avg       0.71      0.75      0.72      1758
weighted avg       0.79      0.76      0.77      1758

ROC-AUC XGBoost: 0.8260274972341874


------------------------

## **7. Comparação dos modelos**

In [34]:
resultados = pd.DataFrame({
    'Modelo': ['Random Forest', 'XGBoost'],
    'ROC-AUC': [
        roc_auc_score(y_test, y_proba_rf),
        roc_auc_score(y_test, y_proba_xgb)
    ]
})

resultados

,Modelo,ROC-AUC
0,Random Forest,0.842205
1,XGBoost,0.826027


Embora ambos os modelos apresentem desempenho competitivo, o Random Forest foi selecionado como modelo final por apresentar maior capacidade discriminativa (ROC-AUC = 0.84) e maior recall da classe churn (80%), característica essencial para estratégias de retenção, onde o custo de não identificar um cliente em risco é elevado.

------------------------

## **8. Aprimoramento de performance do modelo vencedor**

### 8.1 Modelo Base

In [35]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

### 8.2 Pipeline

In [36]:
from sklearn.pipeline import Pipeline

pipeline_rf = Pipeline(
    steps=[
        ('preprocessamento', preprocessador),
        ('modelo', rf)
    ]
)

### 8.3 Grid de Hiperparâmetros

In [37]:
param_grid = {
    'modelo__n_estimators': [300, 500],
    'modelo__max_depth': [8, 10, 12],
    'modelo__min_samples_leaf': [10, 20],
    'modelo__min_samples_split': [10, 20],
    'modelo__max_features': ['sqrt']
}

### 8.4 Gridsearch

In [38]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(
    estimator=pipeline_rf,
    param_grid=param_grid,
    scoring='recall',
    cv=5,
    verbose=2,
    n_jobs=-1
)

O GridSearch foi otimizado utilizando a métrica de recall,
priorizando a identificação de clientes churn,
mesmo ao custo de aumento de falsos positivos.

### 8.5 Treinamento

In [39]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...m_state=42))])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'modelo__max_depth': [8, 10, ...], 'modelo__max_features': ['sqrt'], 'modelo__min_samples_leaf': [10, 20], 'modelo__min_samples_split': [10, 20], ...}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'recall'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation 

### 8.6 Melhores Parâmetros

In [40]:
grid.best_params_

{'modelo__max_depth': 8,
 'modelo__max_features': 'sqrt',
 'modelo__min_samples_leaf': 20,
 'modelo__min_samples_split': 10,
 'modelo__n_estimators': 500}

### 8.7 Modelo Otimizado

In [41]:
best_rf = grid.best_estimator_

### 8.8 Avaliação Final

In [42]:
from sklearn.metrics import classification_report, roc_auc_score

y_pred = best_rf.predict(X_test)
y_proba = best_rf.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

              precision    recall  f1-score   support

           0       0.91      0.73      0.81      1291
           1       0.52      0.81      0.63       467

    accuracy                           0.75      1758
   macro avg       0.72      0.77      0.72      1758
weighted avg       0.81      0.75      0.76      1758

ROC-AUC: 0.8429748364977765


O ajuste de hiperparâmetros via GridSearch resultou em uma
melhoria marginal no recall da classe churn, mantendo estabilidade
nas demais métricas.

Isso indica que o modelo baseline já estava próximo do ótimo dado
o conjunto atual de features, sugerindo que ganhos adicionais
devem vir principalmente de engenharia de atributos e não de
ajustes finos de modelo.

Do ponto de vista de negócio, o modelo prioriza
a identificação de clientes em risco, permitindo
ações preventivas de retenção com maior impacto
financeiro do que a simples maximização da acurácia.

## **9. Salvando o modelo**

In [43]:
import joblib

joblib.dump(best_rf, "../models/best_random_forest.pkl")

['../models/best_random_forest.pkl']

### 9.1 Salvando dados de teste (X_test e y_test)

In [44]:
joblib.dump(X_test, "../data/processed/X_test.pkl")
joblib.dump(y_test, "../data/processed/y_test.pkl")

['../data/processed/y_test.pkl']

### 9.2 Salvando métricas

In [45]:
metricas = {
    "roc_auc": roc_auc_score(y_test, y_proba),
    "modelo": "Random Forest Otimizado"
}

joblib.dump(metricas, "../models/metricas_modelo.pkl")

['../models/metricas_modelo.pkl']

### 9.3 Persistência do Modelo

O modelo Random Forest otimizado foi persistido juntamente
com o conjunto de teste, garantindo reprodutibilidade e
permitindo etapas posteriores de ajuste de threshold e
avaliação baseada em custo sem risco de vazamento de dados.